In [ ]:
let leverage ratio = (1.0M / (ratio - 1.0M)) + 1.0M
let k collateral debt l = (debt * collateral) / (l * (l - 1.0M))
let price collateral debt l = (l * debt) / (collateral * (l - 1.0M))

let debtOut ratio collateral debt collateralIn = 
    let l = leverage ratio
    let currentPrice = price collateral debt l
    let excessCollateral = collateral - debt/currentPrice
    let kStart = k collateral debt l
    //let out = debt - (kStart * (l - 1.0M) * l) / (collateral + collateralIn)

    // o = d - (k (l - 1) l)/(c + i) and c + i!=0 and l^2 !=l
    //let out = debt - (kStart * (l - 1.0M) * l) / (collateral + collateralIn)  
    // o = (c d + l (d t + k (-l) + k))/((l - 1) (c + l t)) and (l - 1) (c + l t)!=0 and l!=0
    //let out = (collateral * debt + (l * (debt * collateralIn - kStart * l + kStart))) / ((l - 1.0m) * (collateral + l * collateralIn))
    // o = k/(l^2 (c + t)) - d and l (c + t)!=0
    // let out = kStart / (l * l * (collateral + collateralIn)) - debt
    // // o = k/((l - 1) l (c + t)) - d and (l - 1) l (c + t)!=0
    // let out = kStart / ((l - 1.0M) * l * (collateral + collateralIn)) - debt
    // // o = d - k/((l - 1) l (c + t)) and (l - 1) l (c + t)!=0
    // let out = debt - kStart / ((l - 1.0M) * l * (collateral + collateralIn))
    // // o = d - k/((l - 1) l^2 (c + t)) and (l - 1) l (c + t)!=0
    // let out = debt - kStart / ((l - 1.0M) * l * l * (collateral + collateralIn))
    // // o = (c d (l - 1) + d (l - 1) t - k)/((l - 1) (c + t)) and (l - 1) (c + t)!=0 and l!=0
    // let out = (collateral * debt * (l - 1.0M) + debt * (l - 1.0M) * collateralIn - kStart) / ((l - 1.0M) * (collateral + collateralIn))
    // o = (c d + d t - k)/(c + t) and c + t!=0
    // let out = (collateral * debt + debt * collateralIn - kStart) / (collateral + collateralIn)
    // o = d - k/(l (c + t)) and l (c + t)!=0
    // let out = debt - kStart / (l * (collateral + collateralIn))
    // o = c - k/(d + t) and d + t!=0
    // let out = collateral - kStart / (debt + collateralIn)
    // o = c - (k (l - 1) l)/(d + t) and d + t!=0 and l^2 !=l
    let out = debt - (kStart * (l - 1.0M) * l) / (collateral + collateralIn)   
    // o = (c d + d t - k (l - 1) l)/(c + t) and c + t!=0 and l^2 !=l
    let out = (collateral * debt + debt * collateralIn - kStart * (l - 1.0M) * l) / (collateral + collateralIn)


    let newCollateral = collateral + collateralIn
    let newDebt = debt + l * out
    let newPrice = price newCollateral newDebt l
    let newExcessCollateral = newCollateral - newDebt/newPrice
    let newRatio = (newPrice * newCollateral) / newDebt
    let newL = leverage newRatio
    let newK = k newCollateral (newCollateral * newPrice - newDebt) newL

    {| a = 0
    ; oldL = l
    ; oldK = k collateral debt l
    ; oldPrice = currentPrice
    ; oldExcessCollateral = excessCollateral
    ; collateralIn = collateralIn
    ; debtOut = out * l
    ; newEffectivePrice = out * l / collateralIn
    ; newCollateral = newCollateral
    ; newExcessCollateral = newExcessCollateral
    ; newDebt = newDebt
    ; newL = newL
    ; newK = newK
    ; newPrice = newPrice
    ; newRatio = newRatio
    |}

let o = debtOut 2.0M 2.0M 1000.0M 0.1M
printfn "debtOut: %A" o





debtOut: { a = 0
  collateralIn = 0.1M
  debtOut = 95.23809523809523809523809524M
  newCollateral = 2.1M
  newDebt = 1095.2380952380952380952380952M
  newEffectivePrice = 952.3809523809523809523809524M
  newExcessCollateral = 1.05M
  newK = 1150M
  newL = 2.0M
  newPrice = 1043.0839002267573696145124716M
  newRatio = 2M
  oldExcessCollateral = 1.0M
  oldK = 1000M
  oldL = 2.0M
  oldPrice = 1000M }
